In [ ]:
import re
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk import WordNetLemmatizer

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

Loading the data

In [ ]:
PATH="./News_Category_Dataset_v3.json"
df = pd.read_json(PATH, lines=True)
df.dropna(inplace=True)
df = df.drop(['link', 'authors','date'], axis=1)
df.head()

Combining the Headline and associated short description

In [ ]:
df['full_review'] = df['headline'] + df['short_description']

Merging Similar classes

In [ ]:
df['category'].replace({"HEALTHY LIVING": "WELLNESS",
              "BUSINESS": "FINANCE",
              "MONEY": "FINANCE",
              "PARENTS": "PARENTING",
              "THE WORLDPOST": "WORLD NEWS",
              "WORLDPOST": "WORLD NEWS",
              "STYLE": "STYLE & BEAUTY",
              "GREEN": "ENVIRONMENT",
              "TASTE": "FOOD & DRINK",
              "SCIENCE": "SCIENCE & TECH",
              "TECH": "SCIENCE & TECH",
              "ARTS": "ARTS & CULTURE",
              "CULTURE & ARTS": "ARTS & CULTURE",
              "COLLEGE": "EDUCATION",
              "FIFTY": "OTHER",
              "GOOD NEWS": "OTHER",
              "WEDDING": "OTHER",
              "DIVORCE": "OTHER",
              "MEDIA": "OTHER",
              "WEIRD NEWS": "OTHER",
                        }, inplace=True
            )
df['category'].value_counts()

Dropping classes with less than 5000 samples, to reduce class imbalance

In [ ]:
retain_categories = df[df['full_review']>5000]['category'].values
short_df = df[df['category'].isin(retain_categories)]

Helper functions for data pre-processing

In [ ]:
def clean_numbers(x):
    if bool(re.search(r'\d', x)):
        x = re.sub('[0-9]{5,}', '#####', x)
        x = re.sub('[0-9]{4}', '####', x)
        x = re.sub('[0-9]{3}', '###', x)
        x = re.sub('[0-9]{2}', '##', x)
    return x

contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}
def _get_contractions(contraction_dict):
    contraction_re = re.compile('(%s)' % '|'.join(contraction_dict.keys()))
    return contraction_dict, contraction_re

contractions, contractions_re = _get_contractions(contraction_dict)
def replace_contractions(text):
    def replace(match):
        return contractions[match.group(0)]
    return contractions_re.sub(replace, text)

Data Pre-processing

In [ ]:
lem = WordNetLemmatizer()

def text_preprocessing(dataFrame):
    """
    Performing tasks such as tokenization, stop words, numbers, punctuations, and empty strings removal, stemming,
    and lemmatization
    """
    
    for index, row in dataFrame.iterrows():
        tokenized = nltk.word_tokenize(row['full_review'].lower()) # Word Tokenization
        punctuation_stopwords_removed = [re.sub('[^\w\s]', '', token) for token in tokenized if not token in stop_words] # punctuations and stopwords removal
        number_removed = list(map(clean_numbers, punctuation_stopwords_removed)) # numbers removal
        contraction_removed = list(map(replace_contractions, number_removed))   # Expanding contractions
        head_desc_lemmatized = [lem.lemmatize(token) for token in contraction_removed] # Word Lemmatization
        empty_str_removed = [token for token in head_desc_lemmatized if token != ''] # empty strings removal

        dataFrame['full_review'][index] = " ".join(empty_str_removed)

In [ ]:
text_preprocessing(short_df)
short_df.to_csv("/content/drive/MyDrive/CSE_448/Final/pre_processed_data.csv", index=False)

In [ ]:
id2label = {
"POLITICS": 0,
"WELLNESS": 1,
"ENTERTAINMENT": 2,
"PARENTING": 3,
"STYLE & BEAUTY" : 4,
"OTHER": 5,
"TRAVEL": 6,
"WORLD NEWS": 7,
"FOOD & DRINK": 8,
"FINANCE": 9,
"SPORTS": 10,
"SCIENCE & TECH": 11,
"ENVIRONMENT": 12,
"ARTS & CULTURE" : 13,
"CRIME" : 14,
"RELIGION": 15
}

label2id = {}
for key in id2label.keys():
  label2id[id2label[key]] = key

For transformers, converting data to transformer format

In [ ]:
short_df['labels'] = short_df['category'].map(id2label)
short_df.to_csv("/content/drive/MyDrive/CSE_448/Final/transformer_format_data.csv", index=False)

Undersampling

In [ ]:
rows_to_remove = short_df[short_df['labels'] == 0].sample(26000, random_state=1).index
df1 = short_df.drop(rows_to_remove)

rows_to_remove = df1[df1['labels'] == 1].sample(18000, random_state=1).index
df1 = df1.drop(rows_to_remove)

rows_to_remove = df1[df1['labels'] == 2].sample(9500, random_state=1).index
df1 = df1.drop(rows_to_remove)

rows_to_remove = df1[df1['labels'] == 3].sample(6900, random_state=1).index
df1 = df1.drop(rows_to_remove)

rows_to_remove = df1[df1['labels'] == 4].sample(6000, random_state=1).index
df1 = df1.drop(rows_to_remove)

rows_to_remove = df1[df1['labels'] == 5].sample(5000, random_state=1).index
df1 = df1.drop(rows_to_remove)

rows_to_remove = df1[df1['labels'] == 6].sample(4100, random_state=1).index
df1 = df1.drop(rows_to_remove)

rows_to_remove = df1[df1['labels'] == 7].sample(3000, random_state=1).index
df1 = df1.drop(rows_to_remove)

rows_to_remove = df1[df1['labels'] == 8].sample(3000, random_state=1).index
df1 = df1.drop(rows_to_remove)

rows_to_remove = df1[df1['labels'] == 9].sample(2000, random_state=1).index
df1 = df1.drop(rows_to_remove)

df1['labels'].value_counts()

In [ ]:
df1.to_csv("/content/drive/MyDrive/CSE_448/Final/undersampled_data.csv", index=False)